### Predict Sahel rainfall with linear regression

In this project we work with **C**limate **I**ndex **C**ollection based on **Mo**del **D**ata (CICMoD) data set (https://github.com/MarcoLandtHayen/climate_index_collection). 

Here, we will try to **predict future** Sahel rainfall (lead times 1 / 3 / 6 months) from current and past information (t<=0) of all input features (including PREC_SAHEL) with **CNN/fc** models:

- Prepare inputs and targets.
- Set up model.
- Evaluate model performance.

**Note:** We start with predicting future Sahel rainfall from its own history alone, hence with **univariate** inputs. Then, we add further input features to have **multivariate** inputs. And ultimately, we add **months as additional input features**.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from json import dump, load
from pathlib import Path

### Import additional functions:
from predict_sahel_rainfall.plot import bar_color
from predict_sahel_rainfall.preprocessing import prepare_inputs_and_target
from predict_sahel_rainfall.models import set_CNN_fc

### Prepare inputs and targets: Univariate

Load collection of climate indices directly from GitHub release.
Use the complete preprocessing pipeline function.

In [8]:
## Set common parameters (except ESN and lead time) for data preprocessing:

# Set url to csv file containing CICMoD indices from desired release:
data_url = (
    "https://github.com/MarcoLandtHayen/climate_index_collection/"
    "releases/download/v2023.03.29.1/climate_indices.csv"
)

# Select target index:
target_index = 'PREC_SAHEL'

# Select all input features:
input_features = [
    'AMO', 'ENSO_12', 'ENSO_3', 'ENSO_34', 'ENSO_4', 'NAO_PC', 'NAO_ST', 
    'NP', 'PDO_PC', 'PREC_SAHEL', 'SAM_PC', 'SAM_ZM', 'SAT_N_ALL', 'SAT_N_LAND',
    'SAT_N_OCEAN', 'SAT_S_ALL', 'SAT_S_LAND', 'SAT_S_OCEAN', 'SOI',
    'SSS_ENA', 'SSS_NA', 'SSS_SA', 'SSS_WNA', 'SST_ESIO', 'SST_HMDR',
    'SST_MED', 'SST_TNA', 'SST_TSA', 'SST_WSIO'
]

# # Select subset of input features:
# input_features = [
#     'PREC_SAHEL',
# ]

# Choose, whether to add months as one-hot encoded features:
add_months = True

# Choose, whether to normalize target index:
norm_target = True

# Specify input length:
input_length = 24

# Specify amount of combined training and validation data relative to test data:
train_test_split = 0.9

# Specify relative amount of combined training and validation used for training:
train_val_split = 0.8

## Optionally choose to scale or normalize input features according to statistics from training data:
# 'no': Keep raw input features.
# 'scale_01': Scale input features with min/max scaling to [0,1].
# 'scale_11': Scale input features with min/max scaling to [-1,1].
# 'norm': Normalize input features, hence subtract mean and divide by std dev.
scale_norm = 'scale_01'

In [9]:
# Set parameters for CNN/fc model:
max_pooling_threshold=20
CNN_filters = [10,20]
CNN_kernel_sizes = [5,5]
batch_normalization = True
alpha = 0.3
fc_units = [20,10]
fc_activation = 'sigmoid'
output_activation = 'linear'
CNN_weight_init = 'glorot_uniform'
CNN_bias_init = 'zeros'
fc_weight_init = 'glorot_uniform'
fc_bias_init = 'zeros'
CNN_weight_reg = None
CNN_bias_reg = None
fc_weight_reg = None
fc_bias_reg = None
learning_rate = 0.0002
loss_function = 'mse'

In [12]:
# Set choice of ESMs:
ESMs = ['CESM', 'FOCI']

# Set choice of lead times:
lead_times = [1,3,6]

# Set number of runs per setting:
n_runs = 3

# Set number of training epochs:
n_epochs = 50

# Set batch size:
batch_size = 20

# Get number of input features, depending on whether or not months are addes as additional features:
if add_months:
    n_features = len(input_features) + 12
else:
    n_features = len(input_features)
    
# Check number of input channels:
print('Number of input features:',n_features)

Number of input features: 41


In [13]:
## Initializs storages for loss curves and correlation, dimension (#ESMs, #lead times, #runs, #epochs+1).
## Need #epochs+1, since we want to store results for untrained model plus after each epoch.
train_loss_all = np.zeros((len(ESMs),len(lead_times),n_runs,n_epochs+1))
val_loss_all = np.zeros((len(ESMs),len(lead_times),n_runs,n_epochs+1))
test_loss_all = np.zeros((len(ESMs),len(lead_times),n_runs,n_epochs+1))
train_correl_all = np.zeros((len(ESMs),len(lead_times),n_runs,n_epochs+1))
val_correl_all = np.zeros((len(ESMs),len(lead_times),n_runs,n_epochs+1))
test_correl_all = np.zeros((len(ESMs),len(lead_times),n_runs,n_epochs+1))

## Loop over ESMs:
for m in range(len(ESMs)):
    
    # Get current ESM:
    ESM = ESMs[m]
    
    # Print status:
    print('ESM:',m+1,'of',len(ESMs))

    ## Loop over lead times:
    for l in range(len(lead_times)):
        
        # Get current lead time:
        lead_time = lead_times[l]
        
        # Print status:
        print('  lead time:',l+1,'of',len(lead_times))

        # Prepare inputs and target for current ESM and lead time:
        (
            train_input,
            train_target,
            val_input,
            val_target,
            test_input,
            test_target,
            train_mean,
            train_std,
            train_min,
            train_max,
        ) = prepare_inputs_and_target(    
            data_url=data_url,
            ESM=ESM,
            target_index=target_index,
            input_features=input_features,
            add_months=add_months,
            norm_target=norm_target,
            lead_time=lead_time,
            input_length=input_length,
            train_test_split=train_test_split,
            train_val_split=train_val_split,
            scale_norm=scale_norm,
        )
        
        # Loop over desired number of training runs:
        for r in range(n_runs):
            
            # Print status:
            print('    run:',r+1,'of',n_runs)
            
            # Set up CNN/fc model:
            model = set_CNN_fc(
                input_length=input_length, 
                n_features=n_features, 
                max_pooling_threshold=max_pooling_threshold,
                CNN_filters=CNN_filters, 
                CNN_kernel_sizes=CNN_kernel_sizes, 
                batch_normalization=batch_normalization,
                alpha=alpha,
                fc_units=fc_units, 
                fc_activation=fc_activation,
                output_activation=output_activation,
                CNN_weight_init=CNN_weight_init,
                CNN_bias_init=CNN_bias_init,
                fc_weight_init=fc_weight_init,
                fc_bias_init=fc_bias_init,
                CNN_weight_reg=CNN_weight_reg,
                CNN_bias_reg=CNN_bias_reg,
                fc_weight_reg=fc_weight_reg,
                fc_bias_reg=fc_bias_reg,
                learning_rate=learning_rate, 
                loss_function=loss_function
            )
            
            ### Train model: Epoch-by-epoch
            
            ## Store results for untrained model:
            
            # Get model predictions on training, validation and test data:
            train_pred = model.predict(train_input)
            val_pred = model.predict(val_input)
            test_pred = model.predict(test_input)

            # Compute mse of model predictions vs. true targets:
            train_loss = np.mean((train_target-train_pred)**2)
            val_loss = np.mean((val_target-val_pred)**2)
            test_loss = np.mean((test_target-test_pred)**2)

            # Compute correlation coefficient of model predictions vs. true targets:
            train_correl = np.corrcoef(np.stack([train_target[:,0],train_pred[:,0]]))[0,1]
            val_correl = np.corrcoef(np.stack([val_target[:,0],val_pred[:,0]]))[0,1]
            test_correl = np.corrcoef(np.stack([test_target[:,0],test_pred[:,0]]))[0,1]
            
            # Store results:
            train_loss_all[m,l,r,0] = train_loss
            val_loss_all[m,l,r,0] = val_loss
            test_loss_all[m,l,r,0] = test_loss
            train_correl_all[m,l,r,0] = train_correl
            val_correl_all[m,l,r,0] = val_correl
            test_correl_all[m,l,r,0] = test_correl          
            
            # Loop over epochs:
            for e in range(n_epochs):
                
                # Train model for single epoch:
                history = model.fit(train_input, train_target, epochs=1, batch_size=batch_size, shuffle=True, verbose=0)

                ## Store results after current epoch:
            
                # Get model predictions on training, validation and test data:
                train_pred = model.predict(train_input)
                val_pred = model.predict(val_input)
                test_pred = model.predict(test_input)

                # Compute mse of model predictions vs. true targets:
                train_loss = np.mean((train_target-train_pred)**2)
                val_loss = np.mean((val_target-val_pred)**2)
                test_loss = np.mean((test_target-test_pred)**2)

                # Compute correlation coefficient of model predictions vs. true targets:
                train_correl = np.corrcoef(np.stack([train_target[:,0],train_pred[:,0]]))[0,1]
                val_correl = np.corrcoef(np.stack([val_target[:,0],val_pred[:,0]]))[0,1]
                test_correl = np.corrcoef(np.stack([test_target[:,0],test_pred[:,0]]))[0,1]

                # Store results:
                train_loss_all[m,l,r,e+1] = train_loss
                val_loss_all[m,l,r,e+1] = val_loss
                test_loss_all[m,l,r,e+1] = test_loss
                train_correl_all[m,l,r,e+1] = train_correl
                val_correl_all[m,l,r,e+1] = val_correl
                test_correl_all[m,l,r,e+1] = test_correl          

ESM: 1 of 2
  lead time: 1 of 3
    run: 1 of 3


2023-06-08 09:26:01.926861: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 09:26:01.994785: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


    run: 2 of 3
    run: 3 of 3
  lead time: 2 of 3
    run: 1 of 3
    run: 2 of 3
    run: 3 of 3
  lead time: 3 of 3
    run: 1 of 3
    run: 2 of 3
    run: 3 of 3
ESM: 2 of 2
  lead time: 1 of 3
    run: 1 of 3
    run: 2 of 3
    run: 3 of 3
  lead time: 2 of 3
    run: 1 of 3
    run: 2 of 3
    run: 3 of 3
  lead time: 3 of 3
    run: 1 of 3
    run: 2 of 3
    run: 3 of 3


In [14]:
# ### Store results:

# ## CNN/fc - univariate:

# # Specify model setup:
# setup = 'CNN/fc - univariate'

# # Save loss and correlation results:
# np.save('../results/quickrun_CNN_fc_univariate_train_loss_all.npy', train_loss_all)
# np.save('../results/quickrun_CNN_fc_univariate_val_loss_all.npy', val_loss_all)
# np.save('../results/quickrun_CNN_fc_univariate_test_loss_all.npy', test_loss_all)
# np.save('../results/quickrun_CNN_fc_univariate_train_correl_all.npy', train_correl_all)
# np.save('../results/quickrun_CNN_fc_univariate_val_correl_all.npy', val_correl_all)
# np.save('../results/quickrun_CNN_fc_univariate_test_correl_all.npy', test_correl_all)

# # Store parameters:
# parameters = {
#     "setup": setup,
#     "data_url": data_url,
#     "target_index": target_index,
#     "input_features": input_features,
#     "add_months": add_months,
#     "norm_target": norm_target,
#     "input_length": input_length,
#     "train_test_split": train_test_split,
#     "train_val_split": train_val_split,
#     "train_val_split": train_val_split,
#     "scale_norm": scale_norm,    
#     "max_pooling_threshold": max_pooling_threshold,
#     "CNN_filters": CNN_filters,
#     "CNN_kernel_sizes": CNN_kernel_sizes,
#     "batch_normalization": batch_normalization,
#     "alpha": alpha,
#     "fc_units": fc_units,
#     "fc_activation": fc_activation,
#     "output_activation": output_activation,
#     "CNN_weight_init": CNN_weight_init,
#     "CNN_bias_init": CNN_bias_init,
#     "fc_weight_init": fc_weight_init,
#     "fc_bias_init": fc_bias_init,
#     "CNN_weight_reg": CNN_weight_reg,
#     "CNN_bias_reg": CNN_bias_reg,
#     "fc_weight_reg": fc_weight_reg,
#     "fc_bias_reg": fc_bias_reg,
#     "learning_rate": learning_rate,
#     "loss_function": loss_function,
#     "ESMs": ESMs,
#     "lead_times": lead_times,
#     "n_runs": n_runs,
#     "n_epochs": n_epochs,
#     "batch_size": batch_size,    
# }

# path_to_store_results = Path('../results')
# with open(path_to_store_results / "quickrun_CNN_fc_univariate_parameters.json", "w") as f:
#     dump(parameters, f)

# #######################################
    
# ## CNN/fc - multivariate:

# # Specify model setup:
# setup = 'CNN/fc - multivariate'

# # Save loss and correlation results:
# np.save('../results/quickrun_CNN_fc_multivariate_train_loss_all.npy', train_loss_all)
# np.save('../results/quickrun_CNN_fc_multivariate_val_loss_all.npy', val_loss_all)
# np.save('../results/quickrun_CNN_fc_multivariate_test_loss_all.npy', test_loss_all)
# np.save('../results/quickrun_CNN_fc_multivariate_train_correl_all.npy', train_correl_all)
# np.save('../results/quickrun_CNN_fc_multivariate_val_correl_all.npy', val_correl_all)
# np.save('../results/quickrun_CNN_fc_multivariate_test_correl_all.npy', test_correl_all)

# # Store parameters:
# parameters = {
#     "setup": setup,
#     "data_url": data_url,
#     "target_index": target_index,
#     "input_features": input_features,
#     "add_months": add_months,
#     "norm_target": norm_target,
#     "input_length": input_length,
#     "train_test_split": train_test_split,
#     "train_val_split": train_val_split,
#     "train_val_split": train_val_split,
#     "scale_norm": scale_norm,    
#     "max_pooling_threshold": max_pooling_threshold,
#     "CNN_filters": CNN_filters,
#     "CNN_kernel_sizes": CNN_kernel_sizes,
#     "batch_normalization": batch_normalization,
#     "alpha": alpha,
#     "fc_units": fc_units,
#     "fc_activation": fc_activation,
#     "output_activation": output_activation,
#     "CNN_weight_init": CNN_weight_init,
#     "CNN_bias_init": CNN_bias_init,
#     "fc_weight_init": fc_weight_init,
#     "fc_bias_init": fc_bias_init,
#     "CNN_weight_reg": CNN_weight_reg,
#     "CNN_bias_reg": CNN_bias_reg,
#     "fc_weight_reg": fc_weight_reg,
#     "fc_bias_reg": fc_bias_reg,
#     "learning_rate": learning_rate,
#     "loss_function": loss_function,
#     "ESMs": ESMs,
#     "lead_times": lead_times,
#     "n_runs": n_runs,
#     "n_epochs": n_epochs,
#     "batch_size": batch_size,    
# }

# path_to_store_results = Path('../results')
# with open(path_to_store_results / "quickrun_CNN_fc_multivariate_parameters.json", "w") as f:
#     dump(parameters, f)

# #######################################
    
# ## CNN/fc - multivariate - months as additional input features:

# # Specify model setup:
# setup = 'CNN/fc - multivariate - with months'

# # Save loss and correlation results:
# np.save('../results/quickrun_CNN_fc_multivariate_with_months_train_loss_all.npy', train_loss_all)
# np.save('../results/quickrun_CNN_fc_multivariate_with_months_val_loss_all.npy', val_loss_all)
# np.save('../results/quickrun_CNN_fc_multivariate_with_months_test_loss_all.npy', test_loss_all)
# np.save('../results/quickrun_CNN_fc_multivariate_with_months_train_correl_all.npy', train_correl_all)
# np.save('../results/quickrun_CNN_fc_multivariate_with_months_val_correl_all.npy', val_correl_all)
# np.save('../results/quickrun_CNN_fc_multivariate_with_months_test_correl_all.npy', test_correl_all)

# # Store parameters:
# parameters = {
#     "setup": setup,
#     "data_url": data_url,
#     "target_index": target_index,
#     "input_features": input_features,
#     "add_months": add_months,
#     "norm_target": norm_target,
#     "input_length": input_length,
#     "train_test_split": train_test_split,
#     "train_val_split": train_val_split,
#     "train_val_split": train_val_split,
#     "scale_norm": scale_norm,    
#     "max_pooling_threshold": max_pooling_threshold,
#     "CNN_filters": CNN_filters,
#     "CNN_kernel_sizes": CNN_kernel_sizes,
#     "batch_normalization": batch_normalization,
#     "alpha": alpha,
#     "fc_units": fc_units,
#     "fc_activation": fc_activation,
#     "output_activation": output_activation,
#     "CNN_weight_init": CNN_weight_init,
#     "CNN_bias_init": CNN_bias_init,
#     "fc_weight_init": fc_weight_init,
#     "fc_bias_init": fc_bias_init,
#     "CNN_weight_reg": CNN_weight_reg,
#     "CNN_bias_reg": CNN_bias_reg,
#     "fc_weight_reg": fc_weight_reg,
#     "fc_bias_reg": fc_bias_reg,
#     "learning_rate": learning_rate,
#     "loss_function": loss_function,
#     "ESMs": ESMs,
#     "lead_times": lead_times,
#     "n_runs": n_runs,
#     "n_epochs": n_epochs,
#     "batch_size": batch_size,    
# }

# path_to_store_results = Path('../results')
# with open(path_to_store_results / "quickrun_CNN_fc_multivariate_with_months_parameters.json", "w") as f:
#     dump(parameters, f)

In [7]:
# ### Reload results:

# ## CNN/fc - univariate:

# # Load loss and correlation results:
# train_loss_all = np.load('../results/quickrun_CNN_fc_univariate_train_loss_all.npy')
# val_loss_all = np.load('../results/quickrun_CNN_fc_univariate_val_loss_all.npy')
# test_loss_all = np.load('../results/quickrun_CNN_fc_univariate_test_loss_all.npy',)
# train_correl_all = np.load('../results/quickrun_CNN_fc_univariate_train_correl_all.npy')
# val_correl_all = np.load('../results/quickrun_CNN_fc_univariate_val_correl_all.npy')
# test_correl_all = np.load('../results/quickrun_CNN_fc_univariate_test_correl_all.npy')

# # Load parameters:
# path_to_store_results = Path('../results')
# with open(path_to_store_results / 'quickrun_CNN_fc_univariate_parameters.json', 'r') as f:
#     parameters=load(f)

# ESMs = parameters['ESMs']
# lead_times = parameters['lead_times']
# n_runs = parameters['n_runs']
# n_epochs = parameters['n_epochs']
        

# #######################################
    
# ## CNN/fc - multivariate:

# # Load loss and correlation results:
# train_loss_all = np.load('../results/quickrun_CNN_fc_multivariate_train_loss_all.npy')
# val_loss_all = np.load('../results/quickrun_CNN_fc_multivariate_val_loss_all.npy')
# test_loss_all = np.load('../results/quickrun_CNN_fc_multivariate_test_loss_all.npy',)
# train_correl_all = np.load('../results/quickrun_CNN_fc_multivariate_train_correl_all.npy')
# val_correl_all = np.load('../results/quickrun_CNN_fc_multivariate_val_correl_all.npy')
# test_correl_all = np.load('../results/quickrun_CNN_fc_multivariate_test_correl_all.npy')

# # Load parameters:
# path_to_store_results = Path('../results')
# with open(path_to_store_results / 'quickrun_CNN_fc_multivariate_parameters.json', 'r') as f:
#     parameters=load(f)

# ESMs = parameters['ESMs']
# lead_times = parameters['lead_times']
# n_runs = parameters['n_runs']
# n_epochs = parameters['n_epochs']

# #######################################
    
# ## CNN/fc - multivariate - with months:

# # Load loss and correlation results:
# train_loss_all = np.load('../results/quickrun_CNN_fc_multivariate_with_months_train_loss_all.npy')
# val_loss_all = np.load('../results/quickrun_CNN_fc_multivariate_with_months_val_loss_all.npy')
# test_loss_all = np.load('../results/quickrun_CNN_fc_multivariate_with_months_test_loss_all.npy',)
# train_correl_all = np.load('../results/quickrun_CNN_fc_multivariate_with_months_train_correl_all.npy')
# val_correl_all = np.load('../results/quickrun_CNN_fc_multivariate_with_months_val_correl_all.npy')
# test_correl_all = np.load('../results/quickrun_CNN_fc_multivariate_with_months_test_correl_all.npy')

# # Load parameters:
# path_to_store_results = Path('../results')
# with open(path_to_store_results / 'quickrun_CNN_fc_multivariate_with_months_parameters.json', 'r') as f:
#     parameters=load(f)

# ESMs = parameters['ESMs']
# lead_times = parameters['lead_times']
# n_runs = parameters['n_runs']
# n_epochs = parameters['n_epochs']

### Postprocessing

We now have loss ('mse') and correlation for complete training, validation and test data after each epoch (starting with untrained model as epoch 0).

Next, we aim to find the **minimum loss and correlation on test data** for the **epoch with minimum validation loss**.
This search is done separately for each ESM, lead time and model run.

In a second step, we compute the **mean loss and correlation on test data over all runs**, separately for each ESM and lead time.

In [22]:
## Initializs storages for test loss (min) and correlation (max), where val loss takes its minimum,
## dimension (#ESMs, #lead times, #runs).
test_loss_min = np.zeros((len(ESMs),len(lead_times),n_runs))
test_correl_max = np.zeros((len(ESMs),len(lead_times),n_runs))

## Initializs storages for mean test loss and correlation, averaged over all training runs,
## dimension (#ESMs, #lead times).
test_loss_min_mean = np.zeros((len(ESMs),len(lead_times)))
test_correl_max_mean = np.zeros((len(ESMs),len(lead_times)))

## Loop over ESMs:
for m in range(len(ESMs)):
    
    ## Loop over lead times:
    for l in range(len(lead_times)):
        
        # Loop over desired number of training runs:
        for r in range(n_runs):
            
            # Get epoch with minimum validation loss for current ESM, lead time and training run:
            e_min = np.argmin(val_loss_all[m,l,r])
            
            # Store corresponding test loss and correlation:
            test_loss_min[m,l,r] = test_loss_all[m,l,r,e_min]
            test_correl_max[m,l,r] = test_correl_all[m,l,r,e_min]
            
        # Get mean test loss and correlation over all training runs, for current ESM and lead time:
        test_loss_min_mean[m,l] = np.mean(test_loss_min[m,l])
        test_correl_max_mean[m,l] = np.mean(test_correl_max[m,l])

### Results: Univariate CNN/fc

In [9]:
test_loss_min_mean

array([[1.08345731, 1.09163194, 1.09296049],
       [0.8049299 , 0.82506141, 0.82554797]])

In [10]:
test_correl_max_mean

array([[0.11017516, 0.07629558, 0.07487747],
       [0.16743967, 0.00311626, 0.01481259]])

### Results: Multivariate CNN/fc (without months as additional input features)

In [4]:
test_loss_min_mean

array([[1.02236418, 1.04956418, 1.05474571],
       [0.81395483, 0.82134209, 0.83556669]])

In [6]:
test_correl_max_mean

array([[0.26251132, 0.21423286, 0.20615248],
       [0.11630802, 0.06902116, 0.03968716]])

### Results: Multivariate CNN/fc (with months as additional input features)

In [23]:
test_loss_min_mean

array([[0.97891029, 0.98520856, 1.04293197],
       [0.79110425, 0.82206947, 0.81808267]])

In [24]:
test_correl_max_mean

array([[0.33877961, 0.31896233, 0.23212236],
       [0.20736868, 0.09712759, 0.08144562]])

### Discussion: CNN / fc models - univariate/multivariate (with/without months as additional input features)

Here, we tried to **predict future** Sahel rainfall (lead times 1 / 3 / 6 months) from current and past information (t<=0) of all input features (including PREC_SAHEL) with **CNN/fc** models.

We started with predicting future Sahel rainfall from its own history alone, hence with **univariate** inputs, which gives us only poor results in terms of high loss ('mse') and low correlation of model predictions and true targets.

Adding further input features to have **multivariate** inputs helps to improve prediction accuracy.
And ultimately, we added **months as additional input features**, which gives us the best results.

However, we find better results for models trained on **CESM** data, compared to **FOCI**.
And it appears to be unreasonable, that models trained on univariate inputs outperform models trained on multivariate inputs, at least for FOCI.